# 웹 스크레이핑을 위한 기본 지식

### HTML의 기본 구조

In [1]:
! mkdir htmltest

하위 디렉터리 또는 파일 htmltest이(가) 이미 있습니다.


In [2]:
%%writefile ./htmltest/HTML_example.html 
<!doctype html>
<html>
 <head>
  <meta charset="utf-8">
  <title>이것은 HTML 예제</title>
 </head>
 <body>
  <h1>출간된 책 정보</h1>
  <p id="book_title">이해가 쏙쏙 되는 파이썬</p>
  <p id="author">홍길동</p>
  <p id="publisher">위키북스 출판사</p>
  <p id="year">2018</p>
 </body>
</html>

Overwriting ./htmltest/HTML_example.html


In [3]:
%%writefile ./htmltest/HTML_example2.html 
<!doctype html>
<html>
 <head>
  <meta charset="utf-8">
  <title>이것은 HTML 예제</title>
 </head>
 <body>
  <h1>출간된 책 정보</h1>
  <p>이해가 쏙쏙 되는 파이썬</p>
  <p>홍길동</p>
  <p>위키북스 출판사</p>
  <p>2018</p>
 </body>
</html>

Overwriting ./htmltest/HTML_example2.html


### 웹 페이지의 HTML 소스 갖고 오기

In [4]:
import requests

res = requests.get("https://www.google.co.kr")
res

# 출력값
# <Response [200]>
# 올바르게 요청에 응답받는 경우에는 200번대 코드가 나온다.
# 오류 발생시에는 보통 400, 500번대 코드를 반환하게 되는 것. 500번대는 좀 심각한 오류임.


<Response [200]>

In [5]:
res.text[0:300]
# .text를 붙이면 해당하는 text문을 모두 불러올 수 있다.
# 너무 길어서 [0:300]으로 범위 지정한 것.

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="yC2PThRMPrbDPvSF'

In [6]:
import requests

html = requests.get("https://www.google.co.kr").text
html[0:300]

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="+JtKn40rQ7tYI3vD'

In [7]:
# 잘못된 url 입력하는 경우

# import requests

# html_error = requests.get("https/ww.google.go.kr").text

# 출력값
# MissingSchema: Invalid URL 'https/ww.google.go.kr': No schema supplied. Perhaps you meant http://https/ww.google.go.kr?

In [8]:
type(res), type(html)

(requests.models.Response, str)

### HTML 소스코드를 분석하고 처리하기

**데이터 찾고 추출하기**

In [9]:
from bs4 import BeautifulSoup

# 테스트용 html 코드
html = """
<html><body><div><span>
<a href=http://www.naver.com>naver</a>
<a href=https://www.google.com>google</a>
<a href=http://www.daum.net/>daum</a>
</span></div></body></html>
"""
# html 코드를 BeaurifulSoup이 이해할 수 있는 형식으로 파싱이 필요한데 아래 코드로 형성한다.

# BeautifulSoup를 이용해 HTML 소스를 파싱 
soup = BeautifulSoup(html, 'lxml') # lxml은 모든 마크업랭귀지를 파싱해주는 옵션이다.
soup

<html><body><div><span>
<a href="http://www.naver.com">naver</a>
<a href="https://www.google.com">google</a>
<a href="http://www.daum.net/">daum</a>
</span></div></body></html>

In [10]:
type(soup)
# soup은 나오기는 html형식으로 출력되지만, 실제 type은 bs4.BeautifulSoup이다.
# 이제 처리하기 편해진 것

bs4.BeautifulSoup

In [11]:
print(soup.prettify())

<html>
 <body>
  <div>
   <span>
    <a href="http://www.naver.com">
     naver
    </a>
    <a href="https://www.google.com">
     google
    </a>
    <a href="http://www.daum.net/">
     daum
    </a>
   </span>
  </div>
 </body>
</html>



In [12]:
soup.find('a') # 'a'는 tagname
# find를 쓰면 가장 첫번째 조건을 만족하는 내용만 출력한다.

<a href="http://www.naver.com">naver</a>

In [13]:
soup.find('a').get_text()

# 'a' tag 안에 있는 text문만 가지고 오고 싶은 경우에는 .get_text를 쓴다.

'naver'

In [14]:
soup.find_all('a')
# beaurifulsoup에서 find_all 명령어를 사용할 때도 정규표현식 명령어에서 find를 쓸 때와 같이 리스트에 담아 반환해준다.

# 주의할 점!!!
# find_all로 나온 결과값은 list 내에 담기는 것의 의미를 이해하는 것이 중요하다!!!!!!
# find는 값 자체를 하나 출력해주지만, find_all은 출력값이 하나일지라도 list 형식으로 돌려주므로
# 그 값을 찾으려면 index[0]을 호출해야만 결과값을 볼 수 있는 것이다.

[<a href="http://www.naver.com">naver</a>,
 <a href="https://www.google.com">google</a>,
 <a href="http://www.daum.net/">daum</a>]

**주의할 점!!!(find와 find_all의 차이점)**
- find_all로 나온 결과값은 list 내에 담기는 것의 의미를 이해하는 것이 중요하다!!!!!!
- find는 값 자체를 하나 출력해주지만, find_all은 출력값이 하나일지라도 list 형식으로 돌려주므로 그 값을 찾으려면 index[0]을 호출해야만 결과값을 볼 수 있는 것이다.

In [15]:
site_names = soup.find_all('a')
for site_names in site_names:
    print(site_names.get_text())

naver
google
daum


In [16]:
from bs4 import BeautifulSoup

# 테스트용 HTML 코드
html2 = """
<html>
 <head>
  <title>작품과 작가 모음</title>
 </head>
 <body>
  <h1>책 정보</h1>
  <p class="book_title">토지</p>
  <p class="author">박경리</p>
  
  <p class="book_title">태백산맥</p>
  <p class="author">조정래</p>

  <p class="book_title">감옥으로부터의 사색</p>
  <p class="author">신영복</p>
 </body>
</html>
""" 
# class를 잘 활용해야 한다. class 정보를 찾아가는 것이 훨씬 쉽다.
# head 영역에서 design이나 구성을 정해놓고 나중에 class로 불러오기만 하면 한번에 적용되는 것.
# 실제 파이썬 class변수 같은 느낌?

soup2 = BeautifulSoup(html2, 'lxml')

In [17]:
soup2.title

<title>작품과 작가 모음</title>

In [18]:
soup2.body

<body>
<h1>책 정보</h1>
<p class="book_title">토지</p>
<p class="author">박경리</p>
<p class="book_title">태백산맥</p>
<p class="author">조정래</p>
<p class="book_title">감옥으로부터의 사색</p>
<p class="author">신영복</p>
</body>

In [19]:
soup2.body.h1

<h1>책 정보</h1>

In [20]:
soup2.body.p

<p class="book_title">토지</p>

In [21]:
soup2.find('p')

<p class="book_title">토지</p>

In [22]:
soup2.find_all('p')

[<p class="book_title">토지</p>,
 <p class="author">박경리</p>,
 <p class="book_title">태백산맥</p>,
 <p class="author">조정래</p>,
 <p class="book_title">감옥으로부터의 사색</p>,
 <p class="author">신영복</p>]

In [23]:
soup2.find('p', 'book_title')

<p class="book_title">토지</p>

In [24]:
soup2.find('p', 'author')

<p class="author">박경리</p>

In [25]:
soup2.find_all('p', 'book_title')

[<p class="book_title">토지</p>,
 <p class="book_title">태백산맥</p>,
 <p class="book_title">감옥으로부터의 사색</p>]

In [26]:
soup2.find_all('p', 'author')

[<p class="author">박경리</p>,
 <p class="author">조정래</p>,
 <p class="author">신영복</p>]

In [27]:
from bs4 import BeautifulSoup

soup2 = BeautifulSoup(html2, 'lxml')

book_titles = soup2.find_all('p', 'book_title')
authors = soup2.find_all('p', 'author')

for book_title, author in zip(book_titles, authors):
    print(' - {} / {} '.format(book_title.get_text(), author.get_text()))

 - 토지 / 박경리 
 - 태백산맥 / 조정래 
 - 감옥으로부터의 사색 / 신영복 


In [28]:
? zip

## 웹 사이트에서 데이터 가져오기

### 순위 데이터를 가져오기

**웹사이트 순위**

In [29]:
import requests  
from bs4 import BeautifulSoup 

url = "https://www.alexa.com/topsites/countries/KR"

html_website_ranking = requests.get(url).text
soup_website_ranking = BeautifulSoup(html_website_ranking, 'lxml')

# p 태그의 요소 안에서 a 태그의 요소를 찾음
website_ranking = soup_website_ranking.select('div')

In [30]:
type(website_ranking)

list

In [31]:
website_ranking = soup_website_ranking.find_all('div')
len(website_ranking)

501

In [32]:
website_ranking = soup_website_ranking.find_all('div', 'tr site-listing')
# html 검사로 찾은 영역 더블클릭해서 복사하는 식으로 하면 편함.
len(website_ranking)

50

In [33]:
tmp = website_ranking[0]
tmp

<div class="tr site-listing">
<div class="td number">1</div>
<div class="td DescriptionCell">
<p class="">
<a href="/siteinfo/naver.com">Naver.com</a>
</p>
<div class="description">
 <span class="remainder"></span>
</div>
</div>
<div class="td right"><p>14:11</p></div>
<div class="td right"><p>8.46</p></div>
<div class="td right"><p>9.00%</p></div>
<div class="td right"><p>75,421</p></div>
</div>

In [34]:
tmp1 = tmp.find('div', 'td number')
tmp1

<div class="td number">1</div>

In [35]:
tmp2 = tmp.find('a')
tmp2

<a href="/siteinfo/naver.com">Naver.com</a>

In [36]:
rank = tmp1.get_text()
rank

'1'

In [37]:
site = tmp2.get_text()
site

'Naver.com'

In [38]:
link = tmp2['href']
link

'/siteinfo/naver.com'

In [39]:
Rank = []
Site = []
Link = []

print("[Top Sites in South Korea]")
print("-"*50)
for idx in range(len(website_ranking)):
    tmp = website_ranking[idx]
    tmp1 = tmp.find('div', 'td number')
    tmp2 = tmp.find('a')
    rank = tmp1.get_text()
    site = tmp2.get_text()
    link = tmp2['href']
    
    print('{}위. {} ({})'.format(rank, site, link))
    
    Rank.append(rank)
    Site.append(site)
    Link.append(link)

[Top Sites in South Korea]
--------------------------------------------------
1위. Naver.com (/siteinfo/naver.com)
2위. Youtube.com (/siteinfo/youtube.com)
3위. Google.co.kr (/siteinfo/google.co.kr)
4위. Google.com (/siteinfo/google.com)
5위. Daum.net (/siteinfo/daum.net)
6위. Tistory.com (/siteinfo/tistory.com)
7위. Namu.wiki (/siteinfo/namu.wiki)
8위. Facebook.com (/siteinfo/facebook.com)
9위. Dcinside.com (/siteinfo/dcinside.com)
10위. Wikipedia.org (/siteinfo/wikipedia.org)
11위. Gmarket.co.kr (/siteinfo/gmarket.co.kr)
12위. Twitter.com (/siteinfo/twitter.com)
13위. Ruliweb.com (/siteinfo/ruliweb.com)
14위. Blog.me (/siteinfo/blog.me)
15위. Instagram.com (/siteinfo/instagram.com)
16위. Twitch.tv (/siteinfo/twitch.tv)
17위. 11st.co.kr (/siteinfo/11st.co.kr)
18위. Auction.co.kr (/siteinfo/auction.co.kr)
19위. Baidu.com (/siteinfo/baidu.com)
20위. Inven.co.kr (/siteinfo/inven.co.kr)
21위. Donga.com (/siteinfo/donga.com)
22위. Qq.com (/siteinfo/qq.com)
23위. Tmall.com (/siteinfo/tmall.com)
24위. Amazon.com (/

In [82]:
import requests
from bs4 import BeautifulSoup

url2 = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20181106"

html_website_ranking2 = requests.get(url2).text
# print(html_website_ranking2)
soup_website_ranking2 = BeautifulSoup(html_website_ranking2, 'lxml')

# website_ranking2 = soup_website_ranking2.select('list_ranking')
# website_ranking2 = soup_website_ranking2.select('tbody')
# website_ranking2 = soup_website_ranking2.select('tr')
# website_ranking2 = soup_website_ranking2.select('div')

In [83]:
# website_ranking2[0:300]

In [84]:
type(website_ranking2)

bs4.element.ResultSet

In [85]:
website_ranking2 = soup_website_ranking2.find_all('list_ranking')
len(website_ranking2)

0

In [86]:
website_ranking2[0:300]

[]

In [70]:
website_ranking2 = soup_website_ranking2.find_all('tbody', 'tr')
len(website_ranking2)

0

In [71]:
website_ranking2[0:300]

[]